<a href="https://colab.research.google.com/github/Devi899/Fmml2024/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

1 answer
Yes, averaging the validation accuracy across multiple splits can provide more consistent and reliable results. Here’s why:Reasons for Averaging Validation AccuracyReduction of Variability:Variance: A single validation split might yield results that are influenced by specific characteristics or random fluctuations in that particular subset. Multiple splits average out these variations, giving a more stable estimate of model performance.Overfitting and Underfitting: By using different splits, you can better understand how well your model generalizes to various subsets of the data, reducing the risk of overfitting to a particular validation set.Better Representation of Performance:Generalization: Averaging accuracy across multiple splits provides a better indication of how the model performs on unseen data, reflecting its generalization ability more accurately.Data Distribution: Different splits might capture different aspects of the data distribution, and averaging helps to account for this variability.Robustness:Consistency: Averaging results from multiple splits can make your performance evaluation more robust and less sensitive to the peculiarities of any single validation set

K-Fold Cross-Validation:Process: Divide the data into K folds. Train the model K times, each time using a different fold as the validation set and the remaining K-1 folds as the training set. Average the validation accuracies from all K runs.Example:from sklearn.model_selection import KFold
import numpy as np

# Example data
X = ...  # Features
y = ...  # Labels

kf = KFold(n_splits=5)
accuracies = []

for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Define and train your model
    model = ...  # Replace with your model definition
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

    # Evaluate and record accuracy
    val_accuracy = model.evaluate(X_val, y_val)[1]  # Assuming the accuracy is the second element
    accuracies.append(val_accuracy)

# Average accuracy
average_accuracy = np.mean(accuracies)
print(f"Average Validation Accuracy: {average_accuracy:.2f}")2. Repeated Random Splits:Process: Perform multiple random splits of the data into training and validation sets. Train and evaluate the model for each split, then average the validation accuracies.Example:from sklearn.model_selection import train_test_split

accuracies = []
n_repeats = 10

for _ in range(n_repeats):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

    # Define and train your model
    model = ...  # Replace with your model definition
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

    # Evaluate and record accuracy
    val_accuracy = model.evaluate(X_val, y_val)[1]  # Assuming the accuracy is the second element
    accuracies.append(val_accuracy)

# Average accuracy
average_accuracy = np.mean(accuracies)
print(f"Average Validation Accuracy: {average_accuracy:.2f}")ConclusionAveraging validation accuracy across multiple splits helps to ensure that the reported performance of your model is more reliable and less influenced by the idiosyncrasies of any single validation set. This approach provides a more comprehensive view of how well the model generalizes and helps in making more informed decisions about its performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


2 answer
The size of the training and validation sets significantly impacts the reliability of using the validation set to predict test set accuracy. Here’s how different sizes affect this relationship:Impact of Training Set SizeLarge Training Set:Better Model Performance: A larger training set typically allows the model to learn more about the data distribution, improving its ability to generalize.More Accurate Validation: With more training data, the validation set can provide a better estimate of model performance on unseen data because the model has been exposed to a wider variety of data during training.Small Training Set:Potential Overfitting: With a smaller training set, the model might overfit to the limited data, leading to a validation set that may not accurately reflect the performance on the test set.Validation Reliability: A small training set can lead to high variance in validation results, making it less reliable as an estimate for test set performance.Impact of Validation Set SizeLarge Validation Set:More Reliable Estimates: A larger validation set provides a more stable and accurate estimate of model performance because it better represents the diversity of the data the model has learned.Reduced Variability: With a larger validation set, the accuracy measurements are less susceptible to fluctuations caused by small sample sizes.Small Validation Set:Increased Variability: Small validation sets can lead to high variance in accuracy estimates due to the limited number of examples used to evaluate performance.Potential Bias: If the validation set is not representative of the test set, it can lead to biased estimates of how well the model will perform on the test set.Balancing Training and Validation SetsGeneral Guidelines:Typical Splits: Common practices use splits like 80% training and 20% validation, or k-fold cross-validation to ensure that the model’s performance is evaluated comprehensively.Cross-Validation: Using k-fold cross-validation helps in mitigating the issues related to small validation sets by averaging performance over multiple splits.Practical ConsiderationsValidation Set as a Proxy:The validation set is intended to be a proxy for the test set. Ideally, the validation set should be as representative as possible of the test set to ensure accurate performance estimation.Model Complexity:For complex models with many parameters, a larger training set is crucial to prevent overfitting and ensure that the validation set reflects the model's true performance.Evaluation Metrics:Confidence Intervals: When using smaller validation sets, consider evaluating confidence intervals around performance metrics to account for variability.

ExampleHere’s a simple illustration with code snippets to show how varying the size of the validation set affects performance estimates:from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load dataset
data = load_iris()
X = data.data
y = data.target

# Define function to train and evaluate model
def evaluate_model(train_size, val_size):
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=train_size, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=val_size, random_state=42)
    
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    
    val_predictions = model.predict(X_val)
    test_predictions = model.predict(X_test)
    
    val_accuracy = accuracy_score(y_val, val_predictions)
    test_accuracy = accuracy_score(y_test, test_predictions)
    
    return val_accuracy, test_accuracy

# Experiment with different validation sizes
results = []
for val_size in [0.1, 0.2, 0.3]:
    val_accuracy, test_accuracy = evaluate_model(train_size=0.7, val_size=val_size)
    results.append((val_size, val_accuracy, test_accuracy))
    print(f"Validation Size: {val_size*100}%, Validation Accuracy: {val_accuracy:.2f}, Test Accuracy: {test_accuracy:.2f}")

# Output results
print("\nResults Summary:")
for val_size, val_acc, test_acc in results:
    print(f"Validation Size: {val_size*100}%, Validation Accuracy: {val_acc:.2f}, Test Accuracy: {test_acc:.2f}")ConclusionLarger Validation Sets: Provide more accurate and reliable estimates of performance.Smaller Training Sets: Can lead to overfitting, making the validation accuracy a less reliable predictor of test performance.Balancing: Both training and validation set sizes should be chosen carefully to ensure that the validation set provides a good estimate of test performance while allowing the model to learn effectively.By carefully considering the sizes of your training and validation sets, and using techniques like cross-validation, you can improve the reliability of your performance estimates and ensure that your model generalizes well to unseen data.

3 answer
Choosing the right percentage for the validation set involves balancing several factors to ensure that both the training and validation phases provide reliable and meaningful insights into model performance. While there is no one-size-fits-all answer, here are general guidelines and considerations for reserving an appropriate percentage for the validation set:General GuidelinesCommon Practices:Typical Splits: Common practice often involves splitting the dataset into approximately 80% training and 20% validation, or 90% training and 10% validation. These splits are generally effective in balancing the need for a substantial training set with a sufficiently large validation set.Cross-Validation: Alternatively, k-fold cross-validation can be used, where the dataset is divided into k folds. Each fold serves as a validation set once while the remaining k-1 folds are used for training. This approach provides a robust estimate of model performance across different subsets of the data.Considerations for Choosing Validation Set Size:Dataset Size:Large Datasets: For large datasets (e.g., thousands of samples or more), a smaller percentage (e.g., 10% or 15%) for the validation set might be sufficient, as even a small percentage can represent a substantial number of samples.Small Datasets: For smaller datasets, a larger percentage (e.g., 20% or 30%) might be necessary to ensure the validation set is representative and provides a reliable estimate of model performance.Model Complexity: Complex models with many parameters might benefit from a larger training set to prevent overfitting, which may necessitate a smaller validation set. Conversely, simpler models might allow for a more balanced split.Balancing Training and Validation:Trade-Offs: A larger validation set provides a more stable and reliable estimate of performance but reduces the amount of data available for training. Conversely, a smaller validation set means more data is available for training but may lead to higher variability in performance estimates.Performance Monitoring: Ensure that the validation set is representative of the test set in terms of data distribution, which helps in accurately predicting test set performance.Practical RecommendationsStandard Splits:80/20 Split: Reserve 20% of the data for validation and 80% for training. This is a widely used split that balances the need for a sizable training dataset with a validation set that is large enough to provide reliable performance estimates.90/10 Split: Reserve 10% of the data for validation and 90% for training. This split is often used for very large datasets where a smaller percentage still represents a substantial number of samples.Cross-Validation:k-Fold Cross-Validation: Use k-fold cross-validation with k=5 or k=10, which allows every data point to be used for both training and validation across different folds. This approach helps to mitigate the impact of any single validation set size and provides a robust estimate of model performance.Example ImplementationHere’s a code example showing how to split data for training and validation using an 80/20 split and how to use k-fold cross-validation:from sklearn.model_selection import train_test_split, KFold
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Load dataset
data = load_iris()
X = data.data
y = data.target

# 80/20 Split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)
val_predictions = model.predict(X_val)
val_accuracy = accuracy_score(y_val, val_predictions)
print(f"80/20 Split Validation Accuracy: {val_accuracy:.2f}")

# k-Fold Cross-Validation
kf = KFold(n_splits=10)
accuracies = []

for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    model.fit(X_train, y_train)
    val_predictions = model.predict(X_val)
    accuracy = accuracy_score(y_val, val_predictions)
    accuracies.append(accuracy)

average_accuracy = np.mean(accuracies)
print(f"k-Fold Cross-Validation Average Accuracy: {average_accuracy:.2f}")Conclusion80/20 or 90/10 Splits: These are good starting points, balancing the need for a large training set with a sufficiently large validation set.Cross-Validation: Provides a robust estimate of model performance and is especially useful for smaller datasets.Ultimately, the choice of validation set size should be guided by dataset size, model complexity, and the specific requirements of your task. Regularly reviewing the impact of different splits on your model’s performance will help ensure that the validation set provides a reliable estimate of performance on unseen data.

1 answer
Changing the percentage of the validation set can significantly impact the accuracy metrics and the overall evaluation of your model's performance. Here's how the accuracy of the validation set is affected by increasing or decreasing its percentage:Increasing the Percentage of the Validation SetReduced Training Data:Training Performance: With more data reserved for validation, less data is available for training. This can lead to the model being trained on a smaller sample, which may result in poorer performance on the training data.Generalization: While the training accuracy might decrease due to less training data, the validation accuracy might become a better estimate of generalization performance, provided the validation set is representative.Accuracy Trends:Validation Accuracy: If the validation set becomes too large, it might be a more accurate reflection of the model’s performance on unseen data. However, if the validation set is too large, it can reduce the amount of data available for training, potentially leading to a model that does not learn sufficiently.Example Code:from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load dataset
data = load_iris()
X = data.data
y = data.target

# Increase validation set size
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)
val_predictions = model.predict(X_val)
val_accuracy = accuracy_score(y_val, val_predictions)
print(f"Validation Accuracy with 30% Validation Set: {val_accuracy:.2f}")Reducing the Percentage of the Validation SetIncreased Training Data:Training Performance: With a smaller validation set, more data is available for training, which often helps the model to learn better and can improve training accuracy.Validation Accuracy: The validation accuracy may become less stable and more susceptible to noise because the validation set is smaller and might not be as representative of the overall data distribution.Accuracy Trends:Validation Accuracy: A smaller validation set might not capture the variability of the data well, leading to less reliable performance estimates. The smaller set may not reflect all the nuances of the test data distribution.Example Code:# Reduce validation set size
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)
val_predictions = model.predict(X_val)
val_accuracy = accuracy_score(y_val, val_predictions)
print(f"Validation Accuracy with 10% Validation Set: {val_accuracy:.2f}")Summary of EffectsIncreasing Validation Percentage:Pros: Potentially more accurate estimate of model performance if the validation set is representative.Cons: Reduces the size of the training set, which might lead to lower training accuracy and potentially less effective learning.Decreasing Validation Percentage:Pros: More data available for training, which can improve training performance and model learning.Cons: Validation accuracy may be less stable and reliable due to a smaller validation set that might not fully represent the data distribution.Balancing ActTo balance the trade-offs between training and validation data sizes:Use Cross-Validation: Techniques like k-fold cross-validation help mitigate issues related to validation set size by evaluating the model across multiple splits, providing a more robust estimate of performance.Monitor Performance: Evaluate how different splits affect model performance and choose a validation set size that offers a good compromise between having enough data to train effectively and enough data to evaluate performance reliably.Ultimately, the ideal validation set size depends on the specific dataset and model, and it is often best determined through experimentation and cross-validation.

2 answer
The accuracy of the validation set can indeed provide an estimate of test accuracy, but this estimate’s reliability and accuracy depend on several factors:Impact of Validation Set Size on Accuracy EstimateLarger Validation Set:Pros:Better Representation: A larger validation set is likely to be more representative of the overall data distribution, which can provide a more accurate estimate of how the model will perform on the test set.Stable Estimates: With a larger validation set, the accuracy estimates are generally more stable and less susceptible to fluctuations caused by small sample sizes.Cons:Reduced Training Data: As mentioned earlier, a larger validation set means less data is available for training, which might lead to less effective learning and potentially poorer training accuracy.Smaller Validation Set:Pros:More Training Data: A smaller validation set allows more data for training, which can improve the model’s learning and performance on the training set.Cons:Less Stable Estimates: Smaller validation sets might not capture the variability of the data well, leading to less reliable estimates of performance on unseen data.Higher Variability: The validation accuracy might vary more due to the smaller sample size and may not be as indicative of performance on the test set.Factors Affecting the Accuracy EstimateRepresentativeness:The validation set should ideally be representative of the test set’s distribution to provide a good estimate of test accuracy. If the validation set does not adequately capture the diversity of the data, the estimate might not be accurate.Model Overfitting:If the model is overfitting to the training data, a larger validation set can help detect this issue by providing a more comprehensive evaluation, whereas a smaller validation set might not reveal overfitting as clearly.Cross-Validation:Using cross-validation (e.g., k-fold cross-validation) can provide a more accurate estimate of model performance by averaging results over multiple splits. This approach helps to mitigate the effects of any single validation set’s size and composition.Empirical TestingHere’s how you might test the impact of validation set size on accuracy estimates:from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Load dataset
data = load_iris()
X = data.data
y = data.target

def evaluate_model(validation_size):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=validation_size, random_state=42)
    
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    val_predictions = model.predict(X_val)
    val_accuracy = accuracy_score(y_val, val_predictions)
    
    return val_accuracy

# Test different validation set sizes
validation_sizes = [0.1, 0.2, 0.3]  # 10%, 20%, 30%
accuracies = {}

for size in validation_sizes:
    acc = evaluate_model(size)
    accuracies[size] = acc
    print(f"Validation Size: {size*100}%, Validation Accuracy: {acc:.2f}")

# Compare with k-fold cross-validation (e.g., k=5)
from sklearn.model_selection import KFold

kf = KFold(n_splits=5)
fold_accuracies = []

for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    val_predictions = model.predict(X_val)
    accuracy = accuracy_score(y_val, val_predictions)
    fold_accuracies.append(accuracy)

average_fold_accuracy = np.mean(fold_accuracies)
print(f"Average k-Fold Cross-Validation Accuracy: {average_fold_accuracy:.2f}")SummaryLarger Validation Sets: Typically provide a more accurate estimate of test accuracy as they are more representative and less affected by sample variability.Smaller Validation Sets: Can lead to less reliable estimates and higher variability.Ultimately, balancing the validation set size with the need for adequate training data and using techniques like cross-validation can provide a more robust estimate of model performance on the test set.

3 answer
The number of iterations or repetitions in your model evaluation can significantly impact the accuracy and reliability of performance estimates. Here’s how varying the number of iterations affects the estimate:Effect of Number of IterationsMore Iterations (Repetitions):Increased Stability:Average Performance: More iterations or repetitions typically provide a more stable estimate of model performance by averaging over multiple runs. This helps to smooth out variability due to random fluctuations in individual train-test splits or random initialization.Reduced Variance: By repeating the evaluation multiple times, you reduce the variance in performance estimates. This makes the accuracy estimates more reliable and representative of the model's expected performance on unseen data.Better Estimate of Generalization:Representative Sampling: More iterations help to ensure that different parts of the data are sampled and evaluated, which can give a better estimate of how well the model generalizes to new data.Fewer Iterations:Increased Variability:Higher Variance: Fewer iterations can result in higher variance in performance estimates. A single or a small number of evaluations might not capture the full variability of model performance, leading to potentially misleading conclusions.Less Reliable Estimates: Performance metrics obtained from fewer iterations may not be as representative of the model’s true generalization ability, especially if the dataset is small or if the model's performance is highly sensitive to the specifics of each training set.

Practical ExampleHere’s an example showing how increasing the number of iterations can affect the reliability of performance estimates:from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Load dataset
data = load_iris()
X = data.data
y = data.target

def evaluate_model(n_iterations, test_size=0.2):
    accuracies = []
    for _ in range(n_iterations):
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size, random_state=None)
        model = RandomForestClassifier()
        model.fit(X_train, y_train)
        val_predictions = model.predict(X_val)
        accuracy = accuracy_score(y_val, val_predictions)
        accuracies.append(accuracy)
    
    average_accuracy = np.mean(accuracies)
    return average_accuracy

# Evaluate with different numbers of iterations
iterations_list = [5, 10, 50]
for n_iterations in iterations_list:
    avg_accuracy = evaluate_model(n_iterations)
    print(f"Average Accuracy with {n_iterations} Iterations: {avg_accuracy:.2f}")

# Using cross-validation for comparison
from sklearn.model_selection import KFold

kf = KFold(n_splits=10)
fold_accuracies = []

for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    val_predictions = model.predict(X_val)
    accuracy = accuracy_score(y_val, val_predictions)
    fold_accuracies.append(accuracy)

average_fold_accuracy = np.mean(fold_accuracies)
print(f"Average k-Fold Cross-Validation Accuracy: {average_fold_accuracy:.2f}")SummaryIncreased Iterations:Provide a more stable and reliable estimate of model performance by averaging over multiple runs, thus reducing the impact of random fluctuations.Offer a better representation of the model's generalization ability.Fewer Iterations:Can lead to higher variability and less reliable performance estimates.Cross-Validation:Using cross-validation (e.g., k-fold) is a robust method that inherently includes multiple iterations and offers a comprehensive performance estimate by evaluating the model on multiple different splits of the data.In conclusion, increasing the number of iterations generally improves the reliability and stability of performance estimates, making them more representative of the model’s expected performance on unseen data.

4 answer
Increasing the number of iterations can help mitigate some issues associated with small training or validation datasets, but it does not completely solve the underlying challenges. Here’s how increasing iterations interacts with small datasets and what other strategies you might consider:Impact of Increasing IterationsSmall Training Dataset:Reduced Variability: More iterations can help provide a more stable estimate of the model's performance on small training datasets by averaging out the variability from different random splits.Not a Cure-All: While increasing iterations can improve the stability of performance estimates, it does not compensate for the lack of sufficient data for training. The model might still suffer from overfitting or underfitting due to limited data.Small Validation Dataset:Better Estimates: More iterations with a small validation set can help in providing a more reliable estimate of model performance by reducing the impact of any single split’s variability.Less Effective: However, if the validation set is too small, even multiple iterations might not fully capture the performance variability, and the estimates may still be unreliable.Additional StrategiesCross-Validation:k-Fold Cross-Validation: Use k-fold cross-validation to make the most of the available data. This technique involves splitting the data into k subsets (folds) and using each subset as a validation set while the remaining k-1 subsets are used for training. It provides a more comprehensive estimate of model performance and can be especially useful with small datasets.from sklearn.model_selection import KFold
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

data = load_iris()
X = data.data
y = data.target

kf = KFold(n_splits=5)  # Use 5-fold cross-validation
fold_accuracies = []

for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    val_predictions = model.predict(X_val)
    accuracy = accuracy_score(y_val, val_predictions)
    fold_accuracies.append(accuracy)

average_accuracy = np.mean(fold_accuracies)
print(f"Average k-Fold Cross-Validation Accuracy: {average_accuracy:.2f}")Data Augmentation:Synthetic Data: Use techniques to augment the data, such as generating synthetic examples or using data augmentation methods, to artificially increase the size of the dataset. This can help the model generalize better despite the small dataset.Regularization Techniques:Avoid Overfitting: Apply regularization techniques such as L1/L2 regularization, dropout, or early stopping to prevent the model from overfitting to the small training dataset.Transfer Learning:Pre-Trained Models: Use pre-trained models and fine-tune them on your small dataset. Transfer learning leverages models trained on larger datasets and adapts them to your specific problem.Resampling Techniques:Bootstrapping: Use bootstrapping methods to create multiple samples from the small dataset, allowing for repeated training and validation, which can help in better estimating performance metrics.ConclusionWhile increasing the number of iterations can help stabilize performance estimates when dealing with small datasets, it does not address the fundamental issue of insufficient data. Using strategies like cross-validation, data augmentation, regularization, and transfer learning can provide more effective solutions for improving model performance and reliability with small datasets.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.